In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

# --- 1. Carregar os Dados ---
# Usamos '..' para "voltar" uma pasta (de 'notebooks' para 'Detector de Spam')
# e depois entrar na pasta 'data/'
filepath = '../data/emails.csv'

# Tentar carregar com encoding 'utf-8', se der erro, tentar 'latin-1'
try:
    df = pd.read_csv(filepath, encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv(filepath, encoding='latin-1')

print("--- Dados Carregados ---")
print(df.head()) # Mostra as 5 primeiras linhas
print("\n")
print(df.info()) # Resumo do dataset
print("\n")


# --- 2. Preparar os Dados para o Modelo ---
# O 'X' (features) é o texto do e-mail.
# O 'y' (target) é a coluna que diz se é spam (1) ou não (0).
X = df['text']
y = df['spam']

print(f"Total de e-mails: {len(X)}")
print(f"Total de classes (y): {len(y)}")
print("\n")


# --- 3. Aplicar a Técnica "Bag of Words" ---
# 1. Instanciar o 'vetorizador'. CountVectorizer é a ferramenta do scikit-learn
#    que implementa o Bag of Words.
vectorizer = CountVectorizer()

# 2. 'fit_transform':
#    - 'fit': Aprende todo o vocabulário (todas as palavras únicas) do seu 'X'.
#    - 'transform': Transforma cada e-mail em um vetor (uma linha)
#      onde cada coluna é a contagem de uma palavra do vocabulário.
X_bow = vectorizer.fit_transform(X)

print("--- Bag of Words (BoW) Aplicado ---")
# O resultado é uma "matriz esparsa"
# (Nº de E-mails, Nº de Palavras Únicas no Vocabulário)
print(f"Formato da matriz BoW: {X_bow.shape}")
print("\n")


# --- 4. Dividir em Treino e Teste ---
# Vamos usar 80% dos dados para treinar o modelo e 20% para testar
X_train, X_test, y_train, y_test = train_test_split(
    X_bow,  # Nossos e-mails já vetorizados
    y,      # Nossas classes (spam ou não)
    test_size=0.20, # 20% para teste
    random_state=42 # 'random_state' garante que a divisão seja sempre a mesma
)

print("--- Dados Prontos para Treinamento ---")
print(f"Tamanho do set de Treino (X): {X_train.shape}")
print(f"Tamanho do set de Teste (X): {X_test.shape}")

--- Dados Carregados ---
                                                text  spam
0  Subject: naturally irresistible your corporate...     1
1  Subject: the stock trading gunslinger  fanny i...     1
2  Subject: unbelievable new homes made easy  im ...     1
3  Subject: 4 color printing special  request add...     1
4  Subject: do not have money , get software cds ...     1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5728 entries, 0 to 5727
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5728 non-null   object
 1   spam    5728 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 89.6+ KB
None


Total de e-mails: 5728
Total de classes (y): 5728


--- Bag of Words (BoW) Aplicado ---
Formato da matriz BoW: (5728, 37303)


--- Dados Prontos para Treinamento ---
Tamanho do set de Treino (X): (4582, 37303)
Tamanho do set de Teste (X): (1146, 37303)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

# --- 5. Treinar o Classificador: Naive Bayes ---
# MultinomialNB é o classificador Naive Bayes clássico para contagem de palavras (Bag of Words)
print("--- Treinando Naive Bayes ---")
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)

# Fazer previsões no set de TESTE
y_pred_nb = nb_classifier.predict(X_test)

# --- 6. Avaliar o Naive Bayes ---
accuracy_nb = accuracy_score(y_test, y_pred_nb)
print(f"Acurácia do Naive Bayes: {accuracy_nb * 100:.2f}%")
print("\nRelatório de Classificação (Naive Bayes):")
# Mostra precisão, recall e F1-score
print(classification_report(y_test, y_pred_nb, target_names=['Ham (0)', 'Spam (1)']))
print("-" * 40 + "\n")


# --- 7. Treinar o Classificador: SVM (Support Vector Machine) ---
# Usamos LinearSVC, que é uma implementação de SVM otimizada para texto (muito mais rápida)
print("--- Treinando SVM (LinearSVC) ---")
svm_classifier = LinearSVC(max_iter=5000)
svm_classifier.fit(X_train, y_train)

# Fazer previsões no set de TESTE
y_pred_svm = svm_classifier.predict(X_test)

# --- 8. Avaliar o SVM ---
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"Acurácia do SVM: {accuracy_svm * 100:.2f}%")
print("\nRelatório de Classificação (SVM):")
print(classification_report(y_test, y_pred_svm, target_names=['Ham (0)', 'Spam (1)']))

--- Treinando Naive Bayes ---
Acurácia do Naive Bayes: 99.13%

Relatório de Classificação (Naive Bayes):
              precision    recall  f1-score   support

     Ham (0)       1.00      0.99      0.99       856
    Spam (1)       0.97      0.99      0.98       290

    accuracy                           0.99      1146
   macro avg       0.99      0.99      0.99      1146
weighted avg       0.99      0.99      0.99      1146

----------------------------------------

--- Treinando SVM (LinearSVC) ---
Acurácia do SVM: 98.60%

Relatório de Classificação (SVM):
              precision    recall  f1-score   support

     Ham (0)       0.98      1.00      0.99       856
    Spam (1)       0.99      0.96      0.97       290

    accuracy                           0.99      1146
   macro avg       0.99      0.98      0.98      1146
weighted avg       0.99      0.99      0.99      1146



c:\Users\victo\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
import joblib
import os

# --- 9. Salvar os artefatos do Modelo ---

# Definir o caminho da pasta de modelos
model_dir = '../models'
# Definir os caminhos dos arquivos
model_path = os.path.join(model_dir, 'spam_model.pkl')
vectorizer_path = os.path.join(model_dir, 'vectorizer.pkl')

# Criar a pasta 'models' se ela não existir
os.makedirs(model_dir, exist_ok=True)

# 1. Salvar o 🧠 (O Classificador Naive Bayes)
# Este é o 'cérebro' que aprendeu a diferenciar spam de ham
joblib.dump(nb_classifier, model_path)
print(f"Modelo Naive Bayes salvo em: {model_path}")

# 2. Salvar o 📖 (O Vetorizador Bag of Words)
# Este é o 'dicionário' que transforma texto em números.
# ESTE É O ARQUIVO MAIS IMPORTANTE!
joblib.dump(vectorizer, vectorizer_path)
print(f"Vetorizador Bag of Words salvo em: {vectorizer_path}")

Modelo Naive Bayes salvo em: ../models\spam_model.pkl
Vetorizador Bag of Words salvo em: ../models\vectorizer.pkl


In [ ]:
import joblib

# --- 10. Carregar Modelo e Vetorizador Salvos ---
# (Vamos carregar de novo para ter certeza)
model_path = '../models/spam_model.pkl'
vectorizer_path = '../models/vectorizer.pkl'

loaded_model = joblib.load(model_path)
loaded_vectorizer = joblib.load(vectorizer_path)

print("✅ Modelo e Vetorizador carregados com sucesso!")

# --- 11. Simular Novos E-mails (AGORA EM INGLÊS) ---

# E-mail de teste "Ham" (Não-Spam)
email_teste_ham = [
    "Subject: Hi team, quick update on the project. Please see the attached slides for tomorrow's meeting. Thanks."
]

# E-mail de teste "Spam"
email_teste_spam = [
    "Subject: Congratulations! You won a $1,000,000 lottery prize! Click here to claim your reward NOW. Limited time offer!"
]


# --- 12. Fazer a Previsão ---

print("\n--- Testing 'Ham' Email ---")
# 1. Transformar o texto 'ham'
ham_bow = loaded_vectorizer.transform(email_teste_ham)
# 2. Fazer a previsão
prediction_ham = loaded_model.predict(ham_bow)
# 3. Mostrar o resultado
if prediction_ham[0] == 0:
    print("Result: 0 (Classified as HAM) -> CORRECT!")
else:
    print("Result: 1 (Classified as SPAM) -> INCORRECT!")


print("\n--- Testing 'Spam' Email ---")
# 1. Transformar o texto 'spam'
spam_bow = loaded_vectorizer.transform(email_teste_spam)
# 2. Fazer a previsão
prediction_spam = loaded_model.predict(spam_bow)
# 3. Mostrar o resultado
if prediction_spam[0] == 1:
    print("Result: 1 (Classified as SPAM) -> CORRECT!")
else:
    print("Result: 0 (Classified as HAM) -> INCORRECT!")


✅ Modelo e Vetorizador carregados com sucesso!

--- Testing 'Ham' Email ---
Result: 0 (Classified as HAM) -> CORRECT!

--- Testing 'Spam' Email ---
Result: 1 (Classified as SPAM) -> CORRECT!
